In [1]:
import re
import os
import json
import pandas as pd

In [2]:
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())

In [3]:
persony = pd.read_json(os.path.join("data", "persony_raw.json"))

In [4]:
persony

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
0,Ondřej Mašek (1000),1000,,fyzická,[Vítěz\n Ak...,None,None,None,None,None,None
1,kino Světozor (100019),100019,None,právnická,None,None,None,None,None,None,None
2,"AD WOOD, s.r.o. (100038)",100038,None,právnická,None,None,None,None,None,None,None
3,Hypermarket Film s.r.o. (100039),100039,None,právnická,None,Hypermarket Film,None,None,None,None,None
4,Hypermarket Film s.r.o. (100039),100039,None,právnická,None,Hypermarket Film,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
35452,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,[Vítěz\n Fe...,Josef Voloďa Dvořák (rodné jméno),None,25.04.1942,"Horní Cerekev, Československo",None,None
35453,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,None,27.02.1928,"Mšecké Žehrovice, Československo",27.12.2014,"Bobcaygeon, Kanada"
35454,Čechomor (person),person,None,právnická,[Vítěz\n Ak...,None,None,None,None,None,None
35455,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None,None


In [5]:
def odnebovat(x):
    if x:
        if "nebo" in x:
            x = x.split("nebo")[1].strip()
    return x

In [6]:
def rozdel_misto(x):
    if type(x) == str:
        try:
            mesto = x.split(",")[0]
            stat = x.split(",")[1]
        except:
            mesto = None
            stat = x
    else:
        mesto = None
        stat = None
    return [mesto, stat]

## Doplnění dat z Wikidat

In [7]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18732,Stanislav Litera (35280),35280,,fyzická,None,None,None,None,None,None,None


In [8]:
with open(os.path.join("data_fixes", "data_z_wikidat.json"), encoding="utf-8") as data_z_wikidat:
    data_z_wikidat = json.loads(data_z_wikidat.read())

In [9]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val[0]

In [10]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum úmrtí"] = val[1]

In [11]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18732,Stanislav Litera (35280),35280,,fyzická,None,None,None,11.06.1931,None,21.09.2003,None


## Opravy

In [12]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18337,Eva Pecková (34563),34563,"pedagožka, zpěvačka, tanečnice",fyzická,None,Eva Kubičková (rodné jméno),None,09.12.1941,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika"


In [13]:
for key, val in opravy["data_narozeni"].items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val

In [14]:
for key, val in opravy["jmena"].items():
    persony.loc[persony["Jméno"] == key, "Jméno"] = val

In [15]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18337,Eva Pecková (34563),34563,"pedagožka, zpěvačka, tanečnice",fyzická,None,Eva Kubičková (rodné jméno),None,,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika"


In [16]:
persony[persony["Jméno"] == "Adam Klemens (1779)"]

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
11401,Adam Klemens (1779),1779,"dirigent, hudební skladatel",fyzická,None,None,None,14.01.1967,"Praha, Československo",None,None


## Konverze formátů

In [17]:
persony["Datum narození"].sample(250).unique()

array([None, '1921', '1965', '09.08.2011', '02.04.1941', '03.03.1933',
       '28.09.1986', '15.07.1991', '28.05.1912', '06.06.1969',
       '04.10.1974', '31.05.1973', '19.02.1898', '23.02.1895',
       '27.03.1931', '23.04.1891', '15.02.1923', '02.04.1962',
       '09.05.1978', '1957', '19.05.1971', '17.04.1888', '25.10.1915',
       '23.03.1987', '10.12.1923', '1991', '21.09.1928', '08.05.1868',
       '25.05.1932', '21.01.1988', '22.03.1910', '06.04.1949', '1916',
       '22.12.1998', '01.07.1946', '13.11.1980', '24.03.1955',
       '17.04.1947', '17.08.1904', '17.09.1939', '05.04.1918',
       '03.10.1907', '02.04.1966', '1947', '03.03.1902', '28.06.1950',
       '14.10.1924', '25.03.1942', '20.12.1965', '16.05.1949',
       '15.11.1880', '31.03.1976', '29.12.1962', '04.02.1941',
       '12.04.1907', '02.11.1992', '1939', '24.04.1895', '1950',
       '30.04.1961', '25.11.1895', '08.06.1898', '01.10.1881',
       '03.09.1955', '04.08.1983', '16.01.1947', '1963', '10.07.1936',
     

In [18]:
persony["Město narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[0])
persony["Stát narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[1])
persony["Město úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Stát úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Rok narození"] = pd.to_numeric(persony["Datum narození"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")
persony["Rok úmrtí"] = pd.to_numeric(persony["Datum úmrtí"].str.replace("?","").apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")

In [19]:
def na_data(sloupec):
    global persony
    persony["Pracovni"] = persony[sloupec]
    persony["Pracovni"] = persony["Pracovni"].apply(lambda x: odnebovat(x))
    for index, row in persony.iterrows():
        try:
            date = pd.to_datetime(row["Pracovni"], format = "%d.%m.%Y", errors="ignore")
        except ValueError:
            date = pd.to_datetime(row["Pracovni"], format = "%d. %m. %Y", errors="ignore")
        except ValueError:
            date = pd.to_datetime(row["Pracovni"], format = "%Y", errors="ignore")
        except ValueError:
            date = None
        persony.at[index, sloupec] = date
    persony = persony.drop(["Pracovni"], axis = 1)

In [20]:
na_data("Datum narození")
na_data("Datum úmrtí")

In [21]:
persony["Rok narození"] = pd.to_numeric(persony["Rok narození"], errors="coerce")
persony["Rok úmrtí"] = pd.to_numeric(persony["Rok úmrtí"], errors="coerce")

In [22]:
persony = persony.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [23]:
persony

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození,Rok úmrtí
0,Ondřej Mašek (1000),1000,,fyzická,[Vítěz\n Ak...,None,None,None,None,None,None,None,None,None,None,NaN,NaN
1,kino Světozor (100019),100019,None,právnická,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
2,"AD WOOD, s.r.o. (100038)",100038,None,právnická,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
3,Hypermarket Film s.r.o. (100039),100039,None,právnická,None,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
4,Hypermarket Film s.r.o. (100039),100039,None,právnická,None,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35452,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,[Vítěz\n Fe...,Josef Voloďa Dvořák (rodné jméno),None,1942-04-25 00:00:00,"Horní Cerekev, Československo",None,None,Horní Cerekev,Československo,None,None,1942.0,NaN
35453,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,None,1928-02-27 00:00:00,"Mšecké Žehrovice, Československo",2014-12-27 00:00:00,"Bobcaygeon, Kanada",Mšecké Žehrovice,Československo,Kanada,Kanada,1928.0,2014.0
35454,Čechomor (person),person,None,právnická,[Vítěz\n Ak...,None,None,None,None,None,None,None,None,None,None,NaN,NaN
35455,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN


## Split na ocenění

In [24]:
oceneni = persony[["Jméno","Ocenění"]]

In [25]:
oceneni.to_json(os.path.join("data", "persony_oceneni.json"), orient="records", date_format="iso")

In [26]:
persony = persony.drop(columns="Ocenění")

In [27]:
persony.to_json(os.path.join("data", "persony.json"), orient="records", date_format="iso")